In [1]:
import zipfile
import os

In [2]:
zip_file_path = 'student_images.zip'

In [3]:
extract_to_dir = os.getcwd()

In [4]:
os.makedirs(extract_to_dir, exist_ok=True)

In [5]:
os.listdir('student_images')

['test', 'train', 'validation']

In [6]:
data_dir = os.path.join(extract_to_dir, 'student_images')

In [7]:
!pip install tensorflow keras opencv-python

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data directories
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')
test_dir = os.path.join(data_dir, 'test')

# Create ImageDataGenerator instances
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)


Found 10512 images belonging to 18 classes.
Found 4303 images belonging to 18 classes.
Found 476 images belonging to 18 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [15]:

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)


Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 545s 2s/step - accuracy: 0.3817 - loss: 2.0160 - val_accuracy: 0.8298 - val_loss: 0.5517
Epoch 2/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 521us/step - accuracy: 0.6875 - loss: 0.5402 - val_accuracy: 0.7333 - val_loss: 0.3175
Epoch 3/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 416s 1s/step - accuracy: 0.8157 - loss: 0.5729 - val_accuracy: 0.9293 - val_loss: 0.2522
Epoch 4/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 449us/step - accuracy: 0.8125 - loss: 0.2578 - val_accuracy: 0.9333 - val_loss: 0.0974
Epoch 5/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 413s 1s/step - accuracy: 0.8672 - loss: 0.4144 - val_accuracy: 0.8995 - val_loss: 0.3708
Epoch 6/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 416us/step - accuracy: 0.9062 - loss: 0.1561 - val_accuracy: 0.8667 - val_loss: 0.3075
Epoch 7/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 497s 2s/step - accuracy: 0.9032 - loss: 0.3149 - val_accuracy: 0.9009 - val_loss: 0.2596
Epoch 8/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8438 - loss: 0.1930 - val_ac

In [16]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator, steps=val_generator.samples // val_generator.batch_size)
print(f'Validation Accuracy: {val_accuracy:.2f}')

# Conduct initial testing on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy:.2f}')


134/134 ━━━━━━━━━━━━━━━━━━━━ 36s 267ms/step - accuracy: 0.9485 - loss: 0.1339
Validation Accuracy: 0.95
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 288ms/step - accuracy: 0.9913 - loss: 0.0384
Test Accuracy: 0.99
